In [4]:
#------------------------------HEADER------------------------------#
#Projeto computacional em que implementamos o método simplex.
'''AUTHORS:
Daniel Reis, Matheus Alves, Mariane Santana.
'''

import numpy as np
#------------------------------HEADER------------------------------#

In [5]:
#Função que lê da entrada as informações relevantes para o PL.
def entrada():
    #Lê o tamanho da matriz de restrições
    linhas = int(input("Informe o tamanho da matriz.\nLinhas: "))
    colunas = int(input("\nColunas: "))
    #Cria uma matriz (A)mxn de zeros
    A = np.zeros(linhas*colunas, dtype=float).reshape(linhas, colunas)

    #Atribui às entradas de A os valores do input do usuário.
    for i in range(linhas):
        A[i] = input().split()

    #Atribui às entradas de b (vetor de recursos) os valores do input do usuário.
    recursos = [float(b) for b in input("Informe o vetor dos recursos ("+str(linhas)+" entradas separadas por um espaço): ").split(sep = " ")]

    #Atribui às entradas de c (vetor de custos) os valores do input do usuário.
    custos = [float(c) for c in input("Informe o vetor dos custos ("+str(colunas)+" entradas separadas por um espaço, incluindo zeros para as variáveis de folga): ").split(sep = " ")]

    #Printa os valores lidos para confirmar se tudo correu bem.
    print('A = {}\n'.format(A))
    print('b = {}\n'.format(recursos))
    print('c = {}\n'.format(custos))
    return A, recursos, custos, linhas, colunas


In [6]:
A, recursos, custos, linhas, colunas = entrada()

A = [[10.  2.  1.  1.  0.  0.]
 [ 7.  3.  2.  0.  1.  0.]
 [ 2.  4.  1.  0.  0.  1.]]

b = [100.0, 77.0, 80.0]

c = [-12.0, -3.0, -1.0, 0.0, 0.0, 0.0]



In [81]:

#Função que particiona o PL e cria o vetor de indices das variaveis
def part(A,custos, linhas, colunas):
    # Define partições e os custos básicos e não-básicos
    particao_basica = A[:,(linhas-colunas):].copy()
    particao_nbasica = A[:,:linhas].copy()
    custos_basicos = custos[(linhas-colunas):].copy()
    custos_nbasicos = custos[:linhas].copy()
    vetor_indicial = list(range(1, colunas+1))
    return particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, vetor_indicial

In [132]:

particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, vetor_indicial = part(A,custos, linhas, colunas)

In [83]:
print(particao_basica)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [84]:
print(particao_nbasica)

[[10.  2.  1.]
 [ 7.  3.  2.]
 [ 2.  4.  1.]]


In [133]:
print(vetor_indicial)

[1, 2, 3, 4, 5, 6]


Loop daqui pra baixo

In [118]:
# Procura uma solução básica factível, Xb.
def solucao_basica(particao_basica, recursos):
    try:
        xb = np.linalg.solve(particao_basica, recursos)
        return xb
    except np.linalg.LinAlgError:
        print("Solução básica factível não encontrada.")
        exit(0)
#A funcao acima (solucao_basica) deverá ser chamada em loop até testar todas as particoes possiveis ou encontrar uma solucao factivel?

# def busca_sol_basica(A,recursos):
#     # Define partições e os custos básicos e não-básicos
#     particao_basica = A[:,(linhas-colunas):].copy()
#     particao_nbasica = A[:,:linhas].copy()
#     custos_basicos = custos[(linhas-colunas):].copy()
#     custos_nbasicos = custos[:linhas].copy()
#     #Iterar
#     solucao_basica(particao_basica,recursos)

In [119]:

xb = solucao_basica(particao_basica, recursos)
print(xb)


[100.  77.  80.]


In [120]:
#Função que calcula o valor da função objetivo considerando apenas a parte básica.
def objetivo(custos_basicos, xb):
    valor_atual = np.dot(np.transpose(custos_basicos), xb)
    return valor_atual

In [121]:

valor_atual = objetivo(custos_basicos, xb)
print(valor_atual)

0.0


In [122]:
#Função que calcula os custos relativos (e retorna o quê exatamente? Só os custos?) Ver tbm se a entrada recebe os parametros certos
def relativos(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, linhas, colunas):
    lambda_simplex = np.linalg.solve(np.transpose(particao_basica), custos_basicos)
    custos_relativos = [(custos_nbasicos[j] - np.dot(np.transpose(lambda_simplex), particao_nbasica[:,j])) for j in range(colunas-linhas)]
    
    return custos_relativos

In [123]:
custos_relativos = relativos(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, linhas, colunas)
print(custos_relativos)

[-12.0, -3.0, -1.0]


In [124]:
#Funcao que performa o teste de otimalidade (Essa é tão simples que fico na duvida se cabe implementar direto no main()).
def otima(custos_relativos):
    custo_relativo_min = min(custos_relativos)
    indice_entrada_base = custos_relativos.index(custo_relativo_min)
    if(custo_relativo_min >= 0):
        return (-1)
    else:
        return indice_entrada_base

In [125]:
indice_entrada_base = otima(custos_relativos)
print(indice_entrada_base)

0


In [126]:
# Cálculo da Direção Simplex
direcao_simplex = np.linalg.solve(particao_basica, particao_nbasica[:,indice_entrada_base])
print(direcao_simplex)

[10.  7.  2.]


In [127]:
def indice_saida(xb, particao_basica, particao_nbasica, indice_entrada_base):
    direcao_simplex = np.linalg.solve(particao_basica, particao_nbasica[:,indice_entrada_base])
    for i in list((direcao_simplex <= 0)):
        if i == False:
            passo = xb/direcao_simplex
            epsilon = min(passo)
            indice_saida_base = np.where(passo == epsilon)[0][0]
            return indice_saida_base
    return (-1)


In [128]:
indice_saida_base = indice_saida(xb, particao_basica, particao_nbasica, indice_entrada_base)
print(indice_saida_base)

0


In [134]:
print(vetor_indicial)

[1, 2, 3, 4, 5, 6]


In [135]:
def troca_base(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, indice_entrada_base, indice_saida_base, vetor_indicial, linhas):
    # Realiza troca das colunas da base
    entrada_base = particao_nbasica[:,indice_entrada_base].copy()
    saida_base = particao_basica[:,indice_saida_base].copy()
    particao_basica[:,indice_saida_base] = entrada_base
    particao_nbasica[:,indice_entrada_base] = saida_base

    # Realiza troca dos custos básicos
    entrada_custo = custos_nbasicos[indice_entrada_base]
    saida_custo = custos_basicos[indice_saida_base]
    custos_basicos[indice_saida_base] = entrada_custo
    custos_nbasicos[indice_entrada_base] = saida_custo

    # Realiza atualização do vetor indicial
    entrada_indicial = vetor_indicial[indice_entrada_base]
    saida_indicial = vetor_indicial[indice_saida_base] + linhas
    vetor_indicial[saida_indicial] = entrada_indicial
    vetor_indicial[entrada_indicial] = saida_indicial
    



    return particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, vetor_indicial

In [130]:
particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, vetor_indicial = troca_base(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, indice_entrada_base, indice_saida_base)

In [131]:
print(particao_basica)
print(particao_nbasica)
print(vetor_indicial)

[[10.  0.  0.]
 [ 7.  1.  0.]
 [ 2.  0.  1.]]
[[1. 2. 1.]
 [0. 3. 2.]
 [0. 4. 1.]]
[1, 3, 3, 1, 5, 6]


In [73]:
print(A)

[[10.  2.  1.  1.  0.  0.]
 [ 7.  3.  2.  0.  1.  0.]
 [ 2.  4.  1.  0.  0.  1.]]


In [74]:
print(vetor_indicial)

[3, 4, 2, 0, 1, 5]


In [75]:
print(particao_basica)

[[10.  2.  0.]
 [ 7.  3.  0.]
 [ 2.  4.  1.]]


In [76]:
print(particao_nbasica)

[[1. 0. 1.]
 [0. 1. 2.]
 [0. 0. 1.]]


In [77]:
print(xb)

[ 9.125  4.375 44.25 ]


In [78]:
solucao_basica(particao_nbasica, )

array([ 20., -83.,  80.])

In [136]:
def main():
    
    A, recursos, custos, linhas, colunas = entrada()
    particao_basica, particao_nbasica, custos_basicos, custos_nbasicos = part(A,custos, linhas, colunas)
    xb = solucao_basica(particao_basica, recursos) #Verificar se precisamos iterar sobre todas as possiveis formas de particionar.
    if(min(xb) == None):
        print('Solução básica factível não encontrada.')
        # exit(0) #Termina a execução do programa.

    custos_relativos = relativos(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, linhas, colunas)
    while(otima(custos_relativos) != -1):
        indice_entrada_base = otima(custos_relativos)
        indice_saida_base = indice_saida(xb, particao_basica, particao_nbasica, indice_entrada_base) #direcao de passo
        if indice_saida_base == (-1): #Se direcao é negativa ... O problema é ilimitado.
            print("O problema não tem solução ótima finita.")
            # exit(0) #break;
        else:
            #Funcao que faz a troca (lembrar de informar quais variaveis estao em cada particao na iteracao atual).
            particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, vetor_indicial = troca_base(particao_basica,
                                                                                                particao_nbasica,
                                                                                                custos_basicos,
                                                                                                custos_nbasicos,
                                                                                                indice_entrada_base,
                                                                                                indice_saida_base,
                                                                                                vetor_indicial,
                                                                                                linhas)
            #refaz custo relativo
            custos_relativos = relativos(particao_basica, particao_nbasica, custos_basicos, custos_nbasicos, linhas, colunas)

    if(otima(custos_relativos)):
        valor_atual = objetivo(custos_basicos, xb)

        print("Solução atual é ótima.")
        print("Valor ótimo da solução: ", valor_atual)
        print("Variáveis ótimas: ", xb)
        # exit(0)


    
    

In [137]:
main()

ValueError: could not broadcast input array from shape (0,) into shape (6,)